In [1]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#import tensorflow
import soundfile as sf
import matplotlib.pyplot as plt
import json
from scipy.fftpack import fft, fftfreq
import librosa
import numpy as np
np.set_printoptions(precision=4, suppress=True)
import os
import pandas as pd

## Daten einlesen
siehe librosa_files

In [2]:
# determine path to `src/data`
cwd = os.getcwd()
src_dir = os.path.join(cwd, os.pardir)
data_dir = os.path.join(src_dir, 'data')

# get absolute paths for each file in `src/data`
data_file_paths = map(
    lambda file: os.path.join(data_dir, file),
    os.listdir(data_dir)
)
data_file_paths = list(data_file_paths)

In [8]:
feature_matrix = []
labels = []

for file in data_file_paths:
    extension = os.path.splitext(file)[1]
    if extension == '.flac':
        waveform, sample_rate = librosa.load(os.path.splitext(file)[0] + '.flac')
        label_file = pd.read_csv(os.path.splitext(file)[0] + '.csv')
        
        for index, row in label_file.iterrows():
            feature_matrix.append(fft(waveform[1:1000]))
            labels.append(-1 if (row['label'] == 'No_Whistle') else 1)

In [10]:
print(labels[:20])
print(feature_matrix[:20])

[-1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, -1, -1, 1, -1, -1, 1, -1, 1, -1]
[array([ 0.0017-0.j    ,  0.0114-0.0062j,  0.0082-0.065j ,  0.0226+0.0081j,
        0.0259+0.0586j, -0.0831+0.0452j, -0.0315+0.0092j, -0.1285+0.0794j,
        0.0008+0.0595j,  0.0367+0.0772j, -0.2083-0.0866j,  0.0144-0.0064j,
        0.0515+0.2367j,  0.0575-0.0462j, -0.002 +0.0389j, -0.1153+0.0357j,
       -0.0551-0.0376j, -0.0064-0.0678j, -0.2014-0.0263j,  0.036 -0.0106j,
       -0.0109-0.1103j, -0.0201-0.194j ,  0.1447+0.068j , -0.104 +0.1265j,
       -0.1159-0.0099j, -0.0839-0.0339j,  0.0176+0.3565j, -0.1659+0.2007j,
        0.1187+0.0692j, -0.3583-0.2963j, -0.0437-0.6052j,  0.2259-0.2742j,
        0.2043-0.1887j, -0.0206-0.2757j, -0.3676-0.3791j, -0.5554+0.4708j,
        0.5185-0.2511j,  0.1634+0.4635j, -0.1826+0.1054j, -0.1329+0.0784j,
       -0.1395+0.0579j, -0.0648+0.1412j,  0.2514-0.1035j, -0.2302-0.0686j,
        0.0529-0.3156j,  0.2398-0.111j ,  0.1356+0.0359j, -0.0805+0.022j ,
       -0.0807+0.2308j,  0